In [1]:
!pip install ultralytics opencv-python pyzbar numpy pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00


In [2]:
from google.colab import files
uploaded = files.upload()  # Select your ZIP file

Saving task_1668281_annotations_2025_10_01_14_54_31_yolo 1.1.zip to task_1668281_annotations_2025_10_01_14_54_31_yolo 1.1.zip


In [3]:
!unzip -q *.zip -d dataset
!ls dataset/

obj.data  obj.names  obj_train_data  train.txt


In [4]:
import os
import shutil

# Create new folders
os.makedirs('data/images', exist_ok=True)
os.makedirs('data/labels', exist_ok=True)

# Move files
src_folder = 'dataset/obj_train_data'
for f in os.listdir(src_folder):
    if f.endswith('.jpg') or f.endswith('.png'):
        shutil.copy(os.path.join(src_folder, f), 'data/images/')
    elif f.endswith('.txt'):
        shutil.copy(os.path.join(src_folder, f), 'data/labels/')

print("✅ Images:", len(os.listdir('data/images')))
print("✅ Labels:", len(os.listdir('data/labels')))

✅ Images: 0
✅ Labels: 100


In [5]:
from google.colab import files
print("📤 Please upload ONLY the 100 images you annotated in CVAT.")
print("   Make sure filenames EXACTLY match your .txt label files (e.g., img001.jpg ↔ img001.txt)")
uploaded = files.upload()

📤 Please upload ONLY the 100 images you annotated in CVAT.
   Make sure filenames EXACTLY match your .txt label files (e.g., img001.jpg ↔ img001.txt)


Saving train_images.zip to train_images.zip


In [10]:
# Unzip the full train set
!unzip -q train_images.zip -d all_train_images
print("✅ Unzipped 200 images to 'all_train_images/'")
!ls all_train_images/ | head -5

✅ Unzipped 200 images to 'all_train_images/'
__MACOSX
train_images


In [11]:
import os
import shutil

# Create folder for our 100 images
os.makedirs('my_100_images', exist_ok=True)

# Copy img001.jpg to img100.jpg
for i in range(1, 101):
    src = f'all_train_images/img{i:03d}.jpg'  # → img001.jpg, img002.jpg, ...
    dst = f'my_100_images/img{i:03d}.jpg'
    if os.path.exists(src):
        shutil.copy(src, dst)
    else:
        print(f"⚠️ Missing: {src}")

print(f"✅ Copied {len(os.listdir('my_100_images'))} images")

⚠️ Missing: all_train_images/img001.jpg
⚠️ Missing: all_train_images/img002.jpg
⚠️ Missing: all_train_images/img003.jpg
⚠️ Missing: all_train_images/img004.jpg
⚠️ Missing: all_train_images/img005.jpg
⚠️ Missing: all_train_images/img006.jpg
⚠️ Missing: all_train_images/img007.jpg
⚠️ Missing: all_train_images/img008.jpg
⚠️ Missing: all_train_images/img009.jpg
⚠️ Missing: all_train_images/img010.jpg
⚠️ Missing: all_train_images/img011.jpg
⚠️ Missing: all_train_images/img012.jpg
⚠️ Missing: all_train_images/img013.jpg
⚠️ Missing: all_train_images/img014.jpg
⚠️ Missing: all_train_images/img015.jpg
⚠️ Missing: all_train_images/img016.jpg
⚠️ Missing: all_train_images/img017.jpg
⚠️ Missing: all_train_images/img018.jpg
⚠️ Missing: all_train_images/img019.jpg
⚠️ Missing: all_train_images/img020.jpg
⚠️ Missing: all_train_images/img021.jpg
⚠️ Missing: all_train_images/img022.jpg
⚠️ Missing: all_train_images/img023.jpg
⚠️ Missing: all_train_images/img024.jpg
⚠️ Missing: all_train_images/img025.jpg


In [12]:
# List first 10 image names in the unzipped folder
!ls all_train_images/ | head -10

__MACOSX
train_images


In [13]:
!ls all_train_images/train_images/ | head -5

img001.jpg
img002.jpg
img003.jpg
img004.jpg
img005.jpg


In [14]:
import os
import shutil

# Create clean folder
os.makedirs('my_100_images', exist_ok=True)

# Source folder is now: all_train_images/train_images/
src_folder = 'all_train_images/train_images'

# Copy img001.jpg to img100.jpg
for i in range(1, 101):
    src = f'{src_folder}/img{i:03d}.jpg'
    dst = f'my_100_images/img{i:03d}.jpg'
    if os.path.exists(src):
        shutil.copy(src, dst)
    else:
        print(f"⚠️ Missing: {src}")

print(f"✅ Copied {len(os.listdir('my_100_images'))} images")

✅ Copied 100 images


In [15]:
# Reset data folder
if os.path.exists('data'):
    shutil.rmtree('data')
os.makedirs('data/images', exist_ok=True)
os.makedirs('data/labels', exist_ok=True)

# Copy images
for f in os.listdir('my_100_images'):
    shutil.copy(f'my_100_images/{f}', 'data/images/')

# Copy labels from CVAT
label_dir = 'dataset/obj_train_data'
for f in os.listdir(label_dir):
    if f.endswith('.txt'):
        shutil.copy(f'{label_dir}/{f}', 'data/labels/')

# Verify pairing
img_names = {os.path.splitext(f)[0] for f in os.listdir('data/images')}
lbl_names = {os.path.splitext(f)[0] for f in os.listdir('data/labels')}
common = img_names & lbl_names

print(f"\n✅ Images: {len(img_names)}")
print(f"✅ Labels: {len(lbl_names)}")
print(f"✅ Matched: {len(common)}")

if len(common) == 100:
    print("\n🎉 SUCCESS! You're ready to train YOLOv8.")
else:
    print("\n❌ Still missing some. Check:")
    print("Labels without images:", sorted(lbl_names - img_names))


✅ Images: 100
✅ Labels: 100
✅ Matched: 100

🎉 SUCCESS! You're ready to train YOLOv8.


In [16]:
yaml_content = """
train: /content/data/images
val: /content/data/images

nc: 1
names: ['qr']
"""

with open('data.yaml', 'w') as f:
    f.write(yaml_content)

print("✅ data.yaml created successfully!")

✅ data.yaml created successfully!


In [17]:
from ultralytics import YOLO

# Load pretrained YOLOv8 nano model
model = YOLO('yolov8n.pt')

# Start training
print("🚀 Starting training... (will take 10-20 mins on Colab GPU)")
model.train(
    data='data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,           # reduce to 8 if you get "CUDA out of memory"
    name='qr_detector',
    patience=20         # stop early if no improvement
)

print("✅ Training complete!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
🚀 Starting training... (will take 10-20 mins on Colab GPU)
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0

KeyboardInterrupt: 

In [18]:
!head data/labels/img001.txt

In [19]:
total_qrs = 0
for lbl in os.listdir('data/labels'):
    with open(f'data/labels/{lbl}', 'r') as f:
        total_qrs += len(f.readlines())
print(f"Total QR codes: {total_qrs} in 100 images → avg {total_qrs/100:.2f} per image")

Total QR codes: 6 in 100 images → avg 0.06 per image


In [20]:
import os

non_empty = 0
for lbl in os.listdir('data/labels'):
    if os.path.getsize(f'data/labels/{lbl}') > 0:
        non_empty += 1

print(f"Images with at least 1 QR: {non_empty} / 100")

Images with at least 1 QR: 6 / 100


In [7]:
!unzip -q *.zip -d dataset
!ls dataset/

caution: filename not matched:  train_images.zip
obj.data  obj.names  obj_train_data  train.txt


In [8]:
import os
import shutil

# Clean up old attempts
if os.path.exists('data'):
    shutil.rmtree('data')

# Create fresh folders
os.makedirs('data/images', exist_ok=True)
os.makedirs('data/labels', exist_ok=True)

# 1. Copy uploaded images to data/images/
print("Copying images...")
for filename in os.listdir('/content/'):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        shutil.copy(f'/content/{filename}', 'data/images/')

# 2. Copy labels from CVAT export to data/labels/
print("Copying labels...")
label_dir = 'dataset/obj_train_data'
for filename in os.listdir(label_dir):
    if filename.endswith('.txt'):
        shutil.copy(os.path.join(label_dir, filename), 'data/labels/')

# 3. Verify matching pairs
image_names = {os.path.splitext(f)[0] for f in os.listdir('data/images')}
label_names = {os.path.splitext(f)[0] for f in os.listdir('data/labels')}
common = image_names & label_names

print(f"\n✅ Total images: {len(image_names)}")
print(f"✅ Total labels: {len(label_names)}")
print(f"✅ Matched pairs: {len(common)}")

if len(common) == 100:
    print("🎉 Perfect! All 100 image-label pairs are ready.")
else:
    print("⚠️ Warning: Mismatch detected!")
    print("Images without labels:", sorted(image_names - label_names))
    print("Labels without images:", sorted(label_names - image_names))

Copying images...
Copying labels...

✅ Total images: 0
✅ Total labels: 100
✅ Matched pairs: 0
⚠️ Warning: Mismatch detected!
Images without labels: []
Labels without images: ['img001', 'img002', 'img003', 'img004', 'img005', 'img006', 'img007', 'img008', 'img009', 'img010', 'img011', 'img012', 'img013', 'img014', 'img015', 'img016', 'img017', 'img018', 'img019', 'img020', 'img021', 'img022', 'img023', 'img024', 'img025', 'img026', 'img027', 'img028', 'img029', 'img030', 'img031', 'img032', 'img033', 'img034', 'img035', 'img036', 'img037', 'img038', 'img039', 'img040', 'img041', 'img042', 'img043', 'img044', 'img045', 'img046', 'img047', 'img048', 'img049', 'img050', 'img051', 'img052', 'img053', 'img054', 'img055', 'img056', 'img057', 'img058', 'img059', 'img060', 'img061', 'img062', 'img063', 'img064', 'img065', 'img066', 'img067', 'img068', 'img069', 'img070', 'img071', 'img072', 'img073', 'img074', 'img075', 'img076', 'img077', 'img078', 'img079', 'img080', 'img081', 'img082', 'img0

In [9]:
import os
uploaded_images = [f for f in os.listdir('/content/') if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
print(f"✅ Uploaded {len(uploaded_images)} images")
print("First 5:", uploaded_images[:5])

✅ Uploaded 0 images
First 5: []


In [21]:
from google.colab import files
uploaded = files.upload()  # select your COCO .zip
!unzip -q *.zip -d coco_data

Saving task_1668281_annotations_2025_10_01_15_54_04_coco 1.0.zip to task_1668281_annotations_2025_10_01_15_54_04_coco 1.0.zip
caution: filename not matched:  task_1668281_annotations_2025_10_01_15_54_04_coco 1.0.zip
caution: filename not matched:  train_images.zip


In [22]:
import os
import json
import shutil

# Paths
coco_json = 'coco_data/annotations/instances_default.json'
images_dir = 'all_train_images/train_images'  # your original images
output_dir = 'data_yolo'

# Create output folders
os.makedirs(f'{output_dir}/images', exist_ok=True)
os.makedirs(f'{output_dir}/labels', exist_ok=True)

# Load COCO annotations
with open(coco_json, 'r') as f:
    coco = json.load(f)

# Build image ID → filename map
img_id_to_file = {img['id']: img['file_name'] for img in coco['images']}

# Copy all 100 images (to match labels)
for img_file in os.listdir(images_dir):
    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        shutil.copy(f'{images_dir}/{img_file}', f'{output_dir}/images/')

# Group annotations by image_id
anns_by_image = {}
for ann in coco['annotations']:
    img_id = ann['image_id']
    if img_id not in anns_by_image:
        anns_by_image[img_id] = []
    anns_by_image[img_id].append(ann)

# Convert to YOLO format
total_boxes = 0
for img_id, anns in anns_by_image.items():
    if img_id not in img_id_to_file:
        continue
    img_file = img_id_to_file[img_id]
    img_name = os.path.splitext(img_file)[0]
    label_path = f'{output_dir}/labels/{img_name}.txt'

    # Get image size (needed for normalization)
    img_path = f'{output_dir}/images/{img_file}'
    if not os.path.exists(img_path):
        # Try original source
        img_path = f'{images_dir}/{img_file}'
    from PIL import Image
    with Image.open(img_path) as im:
        img_w, img_h = im.size

    with open(label_path, 'w') as f:
        for ann in anns:
            # COCO bbox: [x_min, y_min, width, height]
            x_min, y_min, w, h = ann['bbox']
            # Convert to YOLO: [x_center, y_center, w, h] (normalized)
            x_center = (x_min + w / 2) / img_w
            y_center = (y_min + h / 2) / img_h
            w_norm = w / img_w
            h_norm = h / img_h
            # Class ID: always 0 (only 'qr' class)
            f.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
            total_boxes += 1

print(f"✅ Converted COCO → YOLO")
print(f"✅ Total QR boxes: {total_boxes}")
print(f"✅ Labels saved to: {output_dir}/labels/")

FileNotFoundError: [Errno 2] No such file or directory: 'coco_data/annotations/instances_default.json'

In [23]:
# List contents of the unzipped folder
!ls -R coco_data/

coco_data/:


In [24]:
!ls -R coco_data/

coco_data/:


In [25]:
from google.colab import files
files.upload()  # select your COCO .zip file

Saving task_1668281_annotations_2025_10_01_15_54_04_coco 1.0.zip to task_1668281_annotations_2025_10_01_15_54_04_coco 1.0 (1).zip


{'task_1668281_annotations_2025_10_01_15_54_04_coco 1.0 (1).zip': b'PK\x03\x04\x14\x00\x00\x00\x00\x00\xa3\x82A[\x03\x98\x92\xdf\xca.\x01\x00\xca.\x01\x00"\x00\x00\x00annotations/instances_default.json{"licenses":[{"name":"","id":0,"url":""}],"info":{"contributor":"","date_created":"","description":"","url":"","version":"","year":""},"categories":[{"id":1,"name":"qr","supercategory":""}],"images":[{"id":1,"width":1484,"height":3264,"file_name":"img001.jpg","license":0,"flickr_url":"","coco_url":"","date_captured":0},{"id":2,"width":3072,"height":4096,"file_name":"img002.jpg","license":0,"flickr_url":"","coco_url":"","date_captured":0},{"id":3,"width":1484,"height":3264,"file_name":"img003.jpg","license":0,"flickr_url":"","coco_url":"","date_captured":0},{"id":4,"width":1864,"height":4096,"file_name":"img004.jpg","license":0,"flickr_url":"","coco_url":"","date_captured":0},{"id":5,"width":1864,"height":4096,"file_name":"img005.jpg","license":0,"flickr_url":"","coco_url":"","date_capture

In [26]:
!unzip -q *.zip -d coco_data

caution: filename not matched:  task_1668281_annotations_2025_10_01_15_54_04_coco 1.0 (1).zip
caution: filename not matched:  task_1668281_annotations_2025_10_01_15_54_04_coco 1.0.zip
caution: filename not matched:  train_images.zip


In [27]:
!ls -R coco_data/

coco_data/:


In [28]:
task_1668281_annotations_2025_10_01_15_54_04_coco 1.0.zip

SyntaxError: invalid syntax (ipython-input-2487714301.py, line 1)

In [29]:
import os
import glob

# Find the COCO zip (with space)
zip_files = [f for f in os.listdir('/content/') if 'coco' in f.lower() and f.endswith('.zip')]
if zip_files:
    old_name = zip_files[0]
    new_name = old_name.replace(' ', '_').replace('(', '').replace(')', '')
    os.rename(old_name, new_name)
    print(f"✅ Renamed: {old_name} → {new_name}")
else:
    print("❌ No COCO zip found")

❌ No COCO zip found


In [30]:
!unzip -q "task_1668281_annotations_2025_10_01_15_54_04_coco 1.0.zip" -d coco_data

In [31]:
!ls -R coco_data/

coco_data/:
annotations

coco_data/annotations:
instances_default.json


In [32]:
import os
import json
import shutil
from PIL import Image

# ✅ Fixed path based on your structure
coco_json = 'coco_data/annotations/instances_default.json'
images_dir = 'all_train_images/train_images'  # your original images
output_dir = 'data_yolo'

# Create output folders
os.makedirs(f'{output_dir}/images', exist_ok=True)
os.makedirs(f'{output_dir}/labels', exist_ok=True)

# Load COCO data
with open(coco_json, 'r') as f:
    coco = json.load(f)

print(f"📊 COCO Dataset:")
print(f"   Images: {len(coco['images'])}")
print(f"   Annotations: {len(coco['annotations'])}")
print(f"   Categories: {[cat['name'] for cat in coco['categories']]}")

# Map image ID to filename
img_id_to_file = {img['id']: img['file_name'] for img in coco['images']}

# Copy all 100 images
for img_file in os.listdir(images_dir):
    if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
        shutil.copy(f'{images_dir}/{img_file}', f'{output_dir}/images/')

# Group annotations by image
from collections import defaultdict
anns_by_image = defaultdict(list)
for ann in coco['annotations']:
    anns_by_image[ann['image_id']].append(ann)

# Convert to YOLO format
total_boxes = 0
for img_id, anns in anns_by_image.items():
    if img_id not in img_id_to_file:
        continue

    img_file = img_id_to_file[img_id]
    img_name = os.path.splitext(img_file)[0]
    label_path = f'{output_dir}/labels/{img_name}.txt'

    # Get image size
    img_path = f'{output_dir}/images/{img_file}'
    if not os.path.exists(img_path):
        img_path = f'{images_dir}/{img_file}'

    if not os.path.exists(img_path):
        print(f"⚠️ Missing image: {img_file}")
        continue

    with Image.open(img_path) as im:
        img_w, img_h = im.size

    with open(label_path, 'w') as f:
        for ann in anns:
            x_min, y_min, w, h = ann['bbox']
            # Normalize to YOLO format
            x_center = (x_min + w / 2) / img_w
            y_center = (y_min + h / 2) / img_h
            w_norm = w / img_w
            h_norm = h / img_h
            f.write(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}\n")
            total_boxes += 1

print(f"\n🎉 Conversion complete!")
print(f"✅ Total QR boxes: {total_boxes}")
print(f"✅ Avg QRs per image: {total_boxes / len(coco['images']):.2f}")

📊 COCO Dataset:
   Images: 100
   Annotations: 359
   Categories: ['qr']

🎉 Conversion complete!
✅ Total QR boxes: 359
✅ Avg QRs per image: 3.59


In [33]:
yaml_content = """
train: /content/data_yolo/images
val: /content/data_yolo/images

nc: 1
names: ['qr']
"""

with open('data.yaml', 'w') as f:
    f.write(yaml_content)

print("✅ data.yaml created")

✅ data.yaml created


In [34]:
from ultralytics import YOLO

print("🚀 Starting training...")
model = YOLO('yolov8n.pt')  # nano = fast + lightweight

model.train(
    data='data.yaml',
    epochs=100,
    imgsz=640,
    batch=16,           # reduce to 8 if OOM
    name='qr_detector_final',
    patience=20         # stop if no improvement
)

print("✅ Training complete!")

🚀 Starting training...
Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=qr_detector_final, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.0, 

KeyboardInterrupt: 

In [35]:
from google.colab import files
files.download('runs/detect/qr_detector_final/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [36]:
from ultralytics import YOLO
model = YOLO('runs/detect/qr_detector_final/weights/best.pt')
metrics = model.val(data='data.yaml')
print(f"✅ Final mAP@0.5: {metrics.box.map50:.3f}")

Ultralytics 8.3.204 🚀 Python-3.12.11 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1223.4±903.9 MB/s, size: 4057.7 KB)
val: Scanning /content/data_yolo/labels.cache... 100 images, 0 backgrounds, 1 corrupt: 100% ━━━━━━━━━━━━ 100/100 20.3Kit/s 0.0s
train: /content/data_yolo/images/img001.jpg: corrupt JPEG restored and saved
train: /content/data_yolo/images/img002.jpg: corrupt JPEG restored and saved
train: /content/data_yolo/images/img003.jpg: corrupt JPEG restored and saved
train: /content/data_yolo/images/img004.jpg: corrupt JPEG restored and saved
train: /content/data_yolo/images/img005.jpg: corrupt JPEG restored and saved
train: /content/data_yolo/images/img006.jpg: corrupt JPEG restored and saved
train: /content/data_yolo/images/img007.jpg: corrupt JPEG restored and saved
train: /content/data_yolo/images/img008.jpg: corrupt JPEG restored and saved
trai

In [37]:
ls -lh weights/best.pt

ls: cannot access 'weights/best.pt': No such file or directory
